# Cloud Workload Optimization - Exploratory Data Analysis

## Overview
This notebook demonstrates comprehensive exploratory data analysis (EDA) of synthetic cloud workload telemetry data, including:
- Data generation and characteristics
- Statistical distributions and correlations
- Temporal patterns and anomalies
- Imbalanced event analysis
- Missing value patterns

**Dataset**: 100K+ synthetic cloud telemetry records  
**Time Period**: 90 days  
**Services**: 20 unique services across 5 clusters

## 1. Setup & Imports

In [ ]:
import sys
import warnings
warnings.filterwarnings('ignore')

# Add project root to path
sys.path.insert(0, '/workspace/src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import logging

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("✓ All imports successful")

## 2. Generate Synthetic Dataset

In [ ]:
from ml_cloud_optimizer.data.generator import CloudWorkloadGenerator, create_sample_dataset

# Generate dataset
logger.info("Generating synthetic cloud workload data...")
df = create_sample_dataset(output_path='../data/raw/cloud_workloads.csv')

# Convert timestamp to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

print(f"\nDataset generated successfully!")
print(f"Shape: {df.shape}")
print(f"Date range: {df['timestamp'].min()} to {df['timestamp'].max()}")
print(f"Duration: {(df['timestamp'].max() - df['timestamp'].min()).days} days")

## 3. Dataset Overview

In [ ]:
# Display basic info
print("="*80)
print("DATASET OVERVIEW")
print("="*80)
print(f"\nShape: {df.shape}")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print("\nData Types:")
print(df.dtypes)

print("\nFirst 5 rows:")
df.head()

In [ ]:
# Statistical summary
print("\nStatistical Summary:")
df.describe()

In [ ]:
# Missing values analysis
print("\nMissing Values:")
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)
missing_df = pd.DataFrame({
    'Column': missing.index,
    'Missing_Count': missing.values,
    'Percentage': missing_pct.values
})
print(missing_df[missing_df['Missing_Count'] > 0])

# Unique values
print(f"\nUnique Services: {df['service_id'].nunique()}")
print(f"Unique Clusters: {df['cluster_id'].nunique()}")

## 4. Distribution Analysis

In [ ]:
# Resource utilization distributions
metrics = ['cpu_utilization', 'memory_utilization', 'network_utilization', 'cost']

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

for idx, metric in enumerate(metrics):
    ax = axes[idx // 2, idx % 2]
    
    # Histogram
    ax.hist(df[metric], bins=50, alpha=0.7, edgecolor='black')
    ax.set_xlabel(metric)
    ax.set_ylabel('Frequency')
    ax.set_title(f'Distribution of {metric}')
    
    # Add statistics
    mean = df[metric].mean()
    median = df[metric].median()
    ax.axvline(mean, color='r', linestyle='--', label=f'Mean: {mean:.3f}')
    ax.axvline(median, color='g', linestyle='--', label=f'Median: {median:.3f}')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../data/processed/distributions.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Distribution plot saved")

In [ ]:
# Box plots for outlier detection
fig, axes = plt.subplots(1, 4, figsize=(16, 4))

for idx, metric in enumerate(metrics):
    ax = axes[idx]
    ax.boxplot(df[metric], vert=True)
    ax.set_ylabel(metric)
    ax.set_title(f'Box plot: {metric}')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../data/processed/boxplots.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Box plot saved")

## 5. Correlation Analysis

In [ ]:
# Calculate correlations
correlation_matrix = df[metrics].corr()

print("Correlation Matrix:")
print(correlation_matrix)

# Heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, fmt='.3f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={'label': 'Correlation'})
plt.title('Correlation Matrix - Resource Metrics')
plt.tight_layout()
plt.savefig('../data/processed/correlation_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ Correlation plot saved")

## 6. Temporal Patterns

In [ ]:
# Extract temporal features
df['hour'] = df['timestamp'].dt.hour
df['day_of_week'] = df['timestamp'].dt.dayofweek
df['date'] = df['timestamp'].dt.date

# Hourly patterns
hourly_stats = df.groupby('hour')[metrics].mean()

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

for idx, metric in enumerate(metrics):
    ax = axes[idx // 2, idx % 2]
    ax.plot(hourly_stats.index, hourly_stats[metric], marker='o', linewidth=2)
    ax.set_xlabel('Hour of Day')
    ax.set_ylabel(metric)
    ax.set_title(f'Hourly Pattern: {metric}')
    ax.grid(True, alpha=0.3)
    ax.set_xticks(range(0, 24, 2))

plt.tight_layout()
plt.savefig('../data/processed/hourly_patterns.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Hourly pattern plot saved")

In [ ]:
# Daily patterns (weekday vs weekend)
day_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
daily_stats = df.groupby('day_of_week')[metrics].mean()
daily_stats.index = [day_names[i] for i in daily_stats.index]

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

for idx, metric in enumerate(metrics):
    ax = axes[idx // 2, idx % 2]
    colors = ['#1f77b4']*5 + ['#ff7f0e']*2  # Blue for weekdays, orange for weekends
    ax.bar(daily_stats.index, daily_stats[metric], color=colors, alpha=0.7, edgecolor='black')
    ax.set_ylabel(metric)
    ax.set_title(f'Daily Pattern: {metric}')
    ax.grid(True, alpha=0.3, axis='y')
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)

plt.tight_layout()
plt.savefig('../data/processed/daily_patterns.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Daily pattern plot saved")

## 7. Imbalanced Event Analysis

In [ ]:
# Scaling events analysis
print("="*80)
print("SCALING EVENTS ANALYSIS (Imbalanced Events)")
print("="*80)

scaling_counts = df['scale_event'].value_counts()
print(f"\nScaling Event Distribution:")
print(f"No Scaling (0): {scaling_counts.get(0, 0)} ({scaling_counts.get(0, 0)/len(df)*100:.2f}%)")
print(f"Scaling (1): {scaling_counts.get(1, 0)} ({scaling_counts.get(1, 0)/len(df)*100:.2f}%)")

# Compare metrics when scaling occurs vs not
comparison = df.groupby('scale_event')[metrics].agg(['mean', 'std', 'min', 'max'])
print("\nMetrics Comparison (Scaling vs No Scaling):")
print(comparison)

In [ ]:
# Visualize scaling events
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=[f'{metric} - Scaling vs No Scaling' for metric in metrics],
    vertical_spacing=0.12,
    horizontal_spacing=0.1
)

colors = ['#1f77b4', '#ff7f0e']  # Blue: no scaling, Orange: scaling
scale_groups = [0, 1]

for idx, metric in enumerate(metrics):
    row = idx // 2 + 1
    col = idx % 2 + 1
    
    for scale_group, color in zip(scale_groups, colors):
        data = df[df['scale_event'] == scale_group][metric]
        fig.add_trace(
            go.Box(
                y=data,
                name=f"Scale={scale_group}",
                marker_color=color,
                boxmean='sd'
            ),
            row=row, col=col
        )

fig.update_layout(height=800, title_text="Resource Metrics Distribution: Scaling vs No Scaling", showlegend=True)
fig.write_html('../data/processed/scaling_comparison.html')
fig.show()

print("✓ Scaling comparison plot saved")

In [ ]:
# Temporal distribution of scaling events
scaling_by_hour = df[df['scale_event'] == 1].groupby('hour').size()
scaling_by_day = df[df['scale_event'] == 1].groupby('day_of_week').size()

fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# By hour
axes[0].bar(scaling_by_hour.index, scaling_by_hour.values, color='#ff7f0e', alpha=0.7, edgecolor='black')
axes[0].set_xlabel('Hour of Day')
axes[0].set_ylabel('Number of Scaling Events')
axes[0].set_title('Scaling Events by Hour of Day')
axes[0].grid(True, alpha=0.3, axis='y')

# By day
day_labels = [day_names[i] for i in scaling_by_day.index]
colors = ['#1f77b4']*5 + ['#ff7f0e']*2
axes[1].bar(day_labels, scaling_by_day.values, color=colors, alpha=0.7, edgecolor='black')
axes[1].set_ylabel('Number of Scaling Events')
axes[1].set_title('Scaling Events by Day of Week')
axes[1].grid(True, alpha=0.3, axis='y')
plt.setp(axes[1].xaxis.get_majorticklabels(), rotation=45)

plt.tight_layout()
plt.savefig('../data/processed/scaling_temporal.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Temporal scaling plot saved")

## 8. Service & Cluster Analysis

In [ ]:
# Top resource consumers
service_stats = df.groupby('service_id').agg({
    'cpu_utilization': 'mean',
    'memory_utilization': 'mean',
    'cost': 'sum',
    'scale_event': 'sum'
}).round(4)
service_stats.columns = ['Avg CPU', 'Avg Memory', 'Total Cost', 'Scaling Events']
service_stats = service_stats.sort_values('Total Cost', ascending=False)

print("Top 10 Resource Consumers (by Cost):")
print(service_stats.head(10))

# Cluster analysis
cluster_stats = df.groupby('cluster_id').agg({
    'cpu_utilization': 'mean',
    'memory_utilization': 'mean',
    'network_utilization': 'mean',
    'cost': 'sum',
    'scale_event': 'sum'
}).round(4)
cluster_stats.columns = ['Avg CPU', 'Avg Memory', 'Avg Network', 'Total Cost', 'Scaling Events']

print("\nCluster-level Statistics:")
print(cluster_stats)

In [ ]:
# Visualize top services
top_10_services = service_stats.head(10)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Cost
axes[0].barh(range(len(top_10_services)), top_10_services['Total Cost'].values, color='#2ca02c', alpha=0.7, edgecolor='black')
axes[0].set_yticks(range(len(top_10_services)))
axes[0].set_yticklabels(top_10_services.index)
axes[0].set_xlabel('Total Cost (USD)')
axes[0].set_title('Top 10 Services by Cost')
axes[0].grid(True, alpha=0.3, axis='x')

# Scaling events
axes[1].barh(range(len(top_10_services)), top_10_services['Scaling Events'].values, color='#ff7f0e', alpha=0.7, edgecolor='black')
axes[1].set_yticks(range(len(top_10_services)))
axes[1].set_yticklabels(top_10_services.index)
axes[1].set_xlabel('Number of Scaling Events')
axes[1].set_title('Top 10 Services by Scaling Events')
axes[1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig('../data/processed/top_services.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Top services plot saved")

## 9. Missing Data Analysis

In [ ]:
# Analyze missing values
print("Missing Value Analysis:")
missing_summary = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum().values,
    'Missing_Percentage': (df.isnull().sum() / len(df) * 100).round(2).values
})
missing_summary = missing_summary[missing_summary['Missing_Count'] > 0]

if len(missing_summary) > 0:
    print(missing_summary)
    
    # Visualize missing data
    fig = go.Figure(data=[
        go.Bar(x=missing_summary['Column'], y=missing_summary['Missing_Percentage'],
               marker_color='#d62728')
    ])
    fig.update_layout(
        title='Missing Data Percentage by Column',
        xaxis_title='Column',
        yaxis_title='Missing Percentage (%)',
        height=400
    )
    fig.write_html('../data/processed/missing_data.html')
    fig.show()
    print("✓ Missing data plot saved")
else:
    print("No missing values after initial cleanup!")

## 10. Key Insights Summary

In [ ]:
print("\n" + "="*80)
print("KEY INSIGHTS FROM EDA")
print("="*80)

insights = f"""
1. DATASET CHARACTERISTICS:
   - Total Records: {len(df):,}
   - Date Range: {df['timestamp'].min().date()} to {df['timestamp'].max().date()}
   - Services: {df['service_id'].nunique()} unique services
   - Clusters: {df['cluster_id'].nunique()} unique clusters

2. RESOURCE UTILIZATION:
   - CPU: Mean={df['cpu_utilization'].mean():.4f}, Std={df['cpu_utilization'].std():.4f}
   - Memory: Mean={df['memory_utilization'].mean():.4f}, Std={df['memory_utilization'].std():.4f}
   - Network: Mean={df['network_utilization'].mean():.4f}, Std={df['network_utilization'].std():.4f}

3. COST ANALYSIS:
   - Total Cost: ${df['cost'].sum():.2f}
   - Average Cost per Record: ${df['cost'].mean():.6f}
   - Cost per Observation: ${df.groupby('timestamp')['cost'].sum().mean():.4f}

4. IMBALANCED EVENTS (Peak Loads):
   - Scaling Events: {df['scale_event'].sum()} out of {len(df):,} ({df['scale_event'].sum()/len(df)*100:.2f}%)
   - Imbalance Ratio: 1:{int(len(df)/(df['scale_event'].sum()+1))}
   - This is a realistic imbalanced classification problem

5. TEMPORAL PATTERNS:
   - Peak Hours: 8-18 (business hours)
   - Weekday vs Weekend: {df[df['day_of_week']<5]['cpu_utilization'].mean():.4f} vs {df[df['day_of_week']>=5]['cpu_utilization'].mean():.4f}
   - Clear diurnal pattern observed

6. CORRELATIONS:
   - CPU-Memory Correlation: {correlation_matrix.loc['cpu_utilization', 'memory_utilization']:.4f} (Strong positive)
   - CPU-Network Correlation: {correlation_matrix.loc['cpu_utilization', 'network_utilization']:.4f}
   - Memory-Cost Correlation: {correlation_matrix.loc['memory_utilization', 'cost']:.4f}

7. TOP RESOURCE CONSUMERS:
   - Top 5 services account for {(service_stats['Total Cost'].head(5).sum() / service_stats['Total Cost'].sum() * 100):.1f}% of total cost
   - Service concentration suggests opportunity for targeted optimization

8. DATA QUALITY:
   - Missing Values: {df.isnull().sum().sum()} total missing values
   - Duplicates: {df.duplicated().sum()} duplicate rows
   - Ready for feature engineering and modeling

9. RECOMMENDATIONS FOR MODELING:
   - Use time-series aware cross-validation (temporal data)
   - Apply SMOTE or class weighting for imbalanced event prediction
   - Include lag features and rolling statistics
   - Consider separate models for peak vs off-peak periods
   - Normalize features (wide range: [0, 1] for CPU/Memory, [0, 100] for Network)
"""

print(insights)

In [ ]:
# Save processed dataset
df_summary = df.copy()
df_summary.to_csv('../data/processed/eda_summary.csv', index=False)

# Save statistics
stats_dict = {
    'Total Records': len(df),
    'Unique Services': df['service_id'].nunique(),
    'Unique Clusters': df['cluster_id'].nunique(),
    'Date Range (Days)': (df['timestamp'].max() - df['timestamp'].min()).days,
    'Scaling Events': int(df['scale_event'].sum()),
    'Scaling Event Rate (%)': round(df['scale_event'].sum()/len(df)*100, 2),
    'Avg CPU': round(df['cpu_utilization'].mean(), 4),
    'Avg Memory': round(df['memory_utilization'].mean(), 4),
    'Total Cost': round(df['cost'].sum(), 2)
}

stats_df = pd.DataFrame([stats_dict]).T
stats_df.columns = ['Value']
stats_df.to_csv('../data/processed/eda_statistics.csv')

print("\n✓ EDA complete! Summary statistics saved.")
print(f"  - eda_summary.csv: Full dataset with temporal features")
print(f"  - eda_statistics.csv: Summary statistics")
print(f"  - Visualizations saved in ../data/processed/")